In [4]:
import pandas as pd
df = pd.read_csv('data/processed/fixed_price.csv')

In [11]:
df['FEE'].value_counts()

FEE
-      565
3       36
6       36
8       36
10      36
15      36
20      36
25      36
30      36
1.5     18
4       18
5       18
Name: count, dtype: int64

In [12]:
df.head()

,SISTEMA,CIA,PRODUCTO,PRODUCTO_CIA,TARIFA,FEE,P1,P2,P3,P4,P5,P6,P1.,P2.,P3.,P4.,P5.,P6.
0,PENINSULA,ACCIONA,FIJO,CIERZO,2.0TD,-,0.088121,0.019570,0.0,0.0,0.0,0.0,0.355703,0.304626,0.265992,0,0,0
1,PENINSULA,ACCIONA,FIJO,LEVANTE,2.0TD,-,0.071682,0.003132,0.0,0.0,0.0,0.0,0.366868,0.315791,0.277157,0,0,0
2,PENINSULA,ACCIONA,FIJO,LEVANTE+,2.0TD,-,0.082641,0.014091,0.0,0.0,0.0,0.0,0.366868,0.315791,0.277157,0,0,0
3,PENINSULA,ACCIONA,FIJO,PONIENTE,2.0TD,-,0.071682,0.003132,0.0,0.0,0.0,0.0,0.360778,0.309701,0.271067,0,0,0
4,PENINSULA,ACCIONA,FIJO,PONIENTE+,2.0TD,-,0.077162,0.008611,0.0,0.0,0.0,0.0,0.360778,0.309701,0.271067,0,0,0


In [ ]:
df.rename(columns={P1.:})

In [7]:
condicion = (df['SISTEMA'] == 'PENINSULA') & (df['TARIFA'] == '2.0TD')
# filtrar para obtener las filas más cercanas por CIA y con SISTEMA y TARIFA específicos

In [20]:
# fixed_price = pd.read_csv('data/processed/fixed_price.csv') #power

In [27]:
# condiciones para filtrar (sería con el desplegable)
condiciones_sistema_tarifa = (index_power['SISTEMA'] == 'PENINSULA') & (index_power['TARIFA'] == '2.0TD') & (index_power['PRODUCTO'] == 'FIJO')

condiciones_cias = index_power['CIA'].isin(['ACCIONA', 'AEQ', 'CANDELA', 'FACTOR', 'IGNIS', 'MAX'])

index_power_filtrado = index_power[condiciones_sistema_tarifa & condiciones_cias] # aplicar el filtro

index_power_filtrado.head()

,SISTEMA,CIA,PRODUCTO,PRODUCTO_CIA,TARIFA,P1,P2,P3,P4,P5,P6
94,PENINSULA,ACCIONA,INDEXADO,CIERZO,2.0TD,0.088121,0.019570,0.0,0.0,0.0,0.0
98,PENINSULA,ACCIONA,INDEXADO,LEVANTE+,2.0TD,0.082641,0.014091,0.0,0.0,0.0,0.0
102,PENINSULA,ACCIONA,INDEXADO,LEVANTE,2.0TD,0.071682,0.003132,0.0,0.0,0.0,0.0
106,PENINSULA,ACCIONA,INDEXADO,PONIENTE+,2.0TD,0.077162,0.008611,0.0,0.0,0.0,0.0
110,PENINSULA,ACCIONA,INDEXADO,PONIENTE,2.0TD,0.071682,0.003132,0.0,0.0,0.0,0.0


In [28]:
filas_mas_cercanas['FEE'] = filas_mas_cercanas['FEE'].str.upper()

In [49]:
# funciones de cálculo importes
def calcular_energia(cons_mens_P1,cons_mens_P2,cons_mens_P3, precio_mens_P1,precio_mens_P2,precio_mens_P3,descuento):
    # sumatorio_cons_mens=cons_mens_P1+cons_mens_P2+cons_mens_P3
    precio_P1_descuento= precio_mens_P1 * (1-descuento) #€
    precio_P2_descuento= precio_mens_P2 * (1-descuento)
    precio_P3_descuento= precio_mens_P3 * (1-descuento)

    total_pago_P1_energia= cons_mens_P1 * precio_P1_descuento #€
    total_pago_P2_energia= cons_mens_P2 * precio_P2_descuento
    total_pago_P3_energia= cons_mens_P3 * precio_P3_descuento

    sumatorio_total_pago_energia = total_pago_P1_energia + total_pago_P2_energia + total_pago_P3_energia
    return sumatorio_total_pago_energia

def calcular_potencia(potencia_contratada_P1,potencia_contratada_P2,dias,precio_potencia_dia_P1,precio_potencia_dia_P2):
    total_pago_P1_potencia= dias * precio_potencia_dia_P1 * potencia_contratada_P1
    total_pago_P2_potencia= dias * precio_potencia_dia_P2 * potencia_contratada_P2
    sumatorio_total_pago_potencia = total_pago_P1_potencia + total_pago_P2_potencia
    return sumatorio_total_pago_potencia

def calcular_total_factura(sumatorio_total_pago_energia,sumatorio_total_pago_potencia,impuesto_electrico,otros,IVA):
    bi_IVA= (sumatorio_total_pago_energia + sumatorio_total_pago_potencia
         +impuesto_electrico + otros)
    importe_total_factura_mens= bi_IVA * (1+IVA)
    return importe_total_factura_mens

def encontrar_opcion_mas_barata(df_energia, df_potencia):
    opciones = []

    df_combinado = pd.merge(df_energia, df_potencia, on='CIA',suffixes=["_E","_P"])

    for index,row in df_combinado.iterrows():
        cons_mens_P1, cons_mens_P2, cons_mens_P3 = 74, 83, 168 #CAMBIAR A INPUT DE LA FACTURA
        potencia_contratada_P1, potencia_contratada_P2 = 3.45, 3.45 #CAMBIAR A INPUT DE LA FACTURA
        precio_mens_P1, precio_mens_P2, precio_mens_P3 = row['P1_E'], row['P2_E'], row['P3_E']
        precio_potencia_dia_P1, precio_potencia_dia_P2 = row['P1_P'], row['P2_P']
        descuento = 0.02  # CAMBIAR A INPUT DE LA FACTURA
        dias = 33 # CAMBIAR A INPUT DE LA FACTURA
        
        sumatorio_total_pago_energia = calcular_energia(cons_mens_P1, cons_mens_P2, cons_mens_P3, precio_mens_P1, precio_mens_P2, precio_mens_P3, descuento)
        sumatorio_total_pago_potencia = calcular_potencia(potencia_contratada_P1, potencia_contratada_P2, dias, precio_potencia_dia_P1, precio_potencia_dia_P2)
        impuesto_electrico, otros, IVA = 5, 2, 0.21  # CAMBIAR A INPUT DE LA FACTURA

        importe_total_factura_mens = calcular_total_factura(sumatorio_total_pago_energia, sumatorio_total_pago_potencia, impuesto_electrico, otros, IVA)

        opciones.append({
            'CIA': row['CIA'],
            'FEE': row['FEE'],
            'PRODUCTO_CIA': row['PRODUCTO_CIA'],
            'CostoTotal': importe_total_factura_mens
        })

    # Opción más barata para cada compañía
    df_opciones = pd.DataFrame(opciones)
    idx_opcion_mas_barata = df_opciones['CostoTotal'].idxmin()
    
    opcion_barata=df_opciones.iloc[idx_opcion_mas_barata]

    opciones_mas_baratas = df_opciones.nsmallest(5, 'CostoTotal')
    
    return 'Opción más barata:',opcion_barata, "Opciones más baratas:", opciones_mas_baratas
    


In [50]:
encontrar_opcion_mas_barata(filas_mas_cercanas,index_power_filtrado)

('Opción más barata:',
 CIA                         FACTOR
 FEE                              4
 PRODUCTO_CIA    DOMESTICO_POOLDBOE
 CostoTotal               77.406676
 Name: 128, dtype: object,
 'Opciones más baratas:',
         CIA  FEE         PRODUCTO_CIA  CostoTotal
 128  FACTOR    4   DOMESTICO_POOLDBOE   77.406676
 130  FACTOR    4  NEGOCIO_POOLNAHORRO   77.406676
 135  FACTOR    6   DOMESTICO_POOLDBOE   78.177446
 137  FACTOR    6  NEGOCIO_POOLNAHORRO   78.177446
 142  FACTOR  7.5   DOMESTICO_POOLDBOE   78.755523)